<a href="https://colab.research.google.com/github/Korniev/Data-Sciense/blob/main/hw_10/hw_10.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Частина 2

В цій частині ми знову будемо працювати з датасетом fasion_mnist.

На відміну від попереднього завдання вам пропонується створити згорткову нейромережу, що використовує VGG16 в якості згорткової основи.

Навчіть отриману мережу на даних із датасету fasion_mnist. Спробуйте досягти максимально можливої точності класифікації за рахунок маніпуляції параметрами мережі. Під час навчання використовуйте прийоми донавчання та виділення ознак.

Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання. Зробіть висновки.

Як завжди, почнемо з імпортів потрібних бібліотек:

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import fashion_mnist
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from keras import optimizers, losses
from sklearn.metrics import classification_report
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

Слідуючи з умови задвання, для наших цілей в якості згорткової основи ми скористаємося мережею VGG16, яка була навчена на даних ImageNet.

Давайте імпортуємо нашу модель, щоб створити основу згортки на базі VGG16:

In [7]:
conv_vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))


Супер, тепер ми можемо розширити нашу модель, додавши кастомні повнозвʼязні шари, для того, щоб наша модель адекватно навчалась на нашому датасеті:

In [9]:
conv_vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
conv_vgg16_base.trainable = False

model = Sequential([
   conv_vgg16_base,
   Flatten(),
   Dense(256, activation="relu"),
   Dense(1, activation="softmax"),
])

Після додавання верхніх шарів, модель потрібно скомпілювати, вказавши оптимізатор, функцію втрат і метрики:

In [10]:
model.compile(
    loss="binary_crossentropy",
    optimizer=optimizers.RMSprop(learning_rate=2e-5),
    metrics=["acc"]
)

Подивимось на архітектуру нашої моделі:

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 2097665 (8.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


Як бачимо, в нас 14714688 не тренерованих параметрів, які ми відключили шляхов додавання рядку conv_vgg16_base.trainable = False